In [11]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

import models as model

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.metrics import mean_squared_error

from sklearn import set_config
from joblib import dump, load

In [12]:
# Note, the orignal score is not here, but is located in other/predictions/920-lasso-predictions-rsme-23328.csv
# Here I tried to keep iterating on the simple model

In [13]:
df_train_raw = pd.read_csv('/home/blue/general-assembly/dsir-824/submissions/projects/project-2-master/datasets/train.csv')

df = df_train_raw.copy()

In [14]:
df_test_raw = pd.read_csv('/home/blue/general-assembly/dsir-824/submissions/projects/project-2-master/datasets/test.csv')
df_test = df_test_raw.copy()

In [15]:
# This will use the entire dataset
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

In [16]:
params = {
    'lasso__alpha': [8, 10, 12],
    'lasso__fit_intercept': [False],
    'lasso__max_iter': [100_000]
},


In [17]:
# Instantiate class
ms = model.ModelSelection(X, y)

In [18]:
pipe = make_pipeline(ms.preprocessing, Lasso())

In [19]:
gs = GridSearchCV(pipe, params, n_jobs=-1)

In [20]:
scores, objects = ms.evaluate_grid_search([gs])

In [21]:
scores

[0.9139818600628189]

In [22]:
objects[0].best_params_

{'lasso__alpha': 12, 'lasso__fit_intercept': False, 'lasso__max_iter': 100000}

In [23]:
#dump(objects[0], '/home/blue/general-assembly/dsir-824/submissions/projects/project-2-master/overfit-')

In [24]:
ms.calculate_rsme(objects)

[21610.55052401455]

In [25]:
# switch to class function

In [26]:
predictions = objects[0].predict(df_test)

In [27]:
predictions = pd.DataFrame(predictions)
predictions = predictions.rename({0:'SalePrice'}, axis=1)
predictions = predictions.join(df_test['Id'])
predictions = predictions[['Id', 'SalePrice']]

In [31]:
#predictions.to_csv('/home/blue/general-assembly/dsir-824/submissions/projects/project-2-master/other/predictions/925-lasso-alpha-20-predictions.csv', index=False)

In [29]:
predictions

,Id,SalePrice
0,2658,118486.784623
1,2718,153721.489749
2,2414,209053.276364
3,1989,111239.529461
4,625,169246.774846
...,...,...
873,1662,195124.871672
874,1234,213141.892805
875,1373,122486.862500
876,1672,111675.315029
